In [1]:

suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(cowplot))


In [2]:
cids <- c("WT","KO","WT_Sham","WT_SNC","KO_Sham","KO_SNC")

sampleNames <- c()
condition <- c()
for (cid in cids){
  for (i in seq(1,3)){
    condition <- c(condition, cid)
    cidi=paste(cid,i, sep="")
    sampleNames <- c(sampleNames, cidi)
  }
}

In [5]:
data <- read.table("results_step2/featureCounts_WT_KO", header=TRUE, quote="\t", skip=1)

n_col <- ncol(data)

names(data)[7:n_col] <- sampleNames
countData <- as.matrix(data[7:n_col])
rownames(countData) <- data$Geneid

# metadata
database <- data.frame(name=sampleNames, condition=as.factor(condition))
rownames(database) <- sampleNames

dds <- DESeqDataSetFromMatrix(countData, colData=database, design= ~ condition)
dds <- dds[ rowSums(counts(dds)) > 1, ]

## call DESeq and get res
dds <- DESeq(dds)


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [6]:
### PCA

rld <- rlog(dds)
p <- plotPCA(rld, intgroup="condition")
ggsave("results_step3/PCA_WT_KO_Sham_SNC.pdf",p)


Saving 6.67 x 6.67 in image



In [9]:
### save normalized counts

normalized_counts <- as.data.frame(counts(dds, normalized=TRUE))

write.csv(normalized_counts, "results_step3/NormalizedCounts_WT_KO_Sham_SNC.csv", row.names=TRUE)


In [11]:
### differential analysis

diff_analysis_list <- list(c("KO","WT"),
                           c("KO_Sham","WT_Sham"),
                           c("WT_SNC","WT_Sham"),
                           c("KO_SNC","KO_Sham"),
                           c("KO_SNC","WT_SNC"),
                           c("KO_SNC","WT_Sham"))

for ( l in diff_analysis_list){
    
    res <- results(dds, contrast=c('condition',l[1],l[2]))
    
    fout_prefix <- "results_step3/diff_analysis_"
    fout <- paste(fout_prefix,l[1],"-vs-",l[2],".csv", sep="")
    
    write.csv(res, fout, row.names=TRUE)

}